In [10]:
%load_ext autoreload
%autoreload 2|

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from tests import TEST_NB
from nb_helpers.utils import *
from nb_helpers.wandb import *

In [12]:
summary_nbs(path=TEST_NB)

CONSOLE.is_terminal(): False

Reading 1 notebooks

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ # ┃ nb name                                   ┃ tracker ┃ wandb features ┃ python libs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 1 │ wandb/nb_helpers/tests/data/test_nb.ipynb │ tracker │                │             │
└───┴───────────────────────────────────────────┴─────────┴────────────────┴─────────────┘

END!

In [13]:
print(TEST_NB)
nb = read_nb(TEST_NB)

/Users/tcapelle/wandb/nb_helpers/tests/data/test_nb.ipynb

In [14]:
nb

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': '# This is a Test NB\n<!--- @wandbcode{tracker} -->'},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': 'this notebooks has code and text and this fancy walala'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': 'import pathlib\ndef f(x): \n    return x'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': 'f(2)'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': 'with open("file.txt", "w") as writer:\n    writer.write(\'one line\')'},
  {'cell_type': 'code',
   'execution_count': None,
   'metadata': {},
   'outputs': [],
   'source': ''}],
 'metadata': {'kernelspec': {'display_name': "Python 3.9.7 64-bit ('tf': conda)",
   'language': 'python',
   'name': 'python3'}},
 'nbformat': 4,
 'nbformat_minor': 2}

In [15]:
def first_cell(nb):
    "return the first cell of the notebook"
    return nb["cells"][0]["source"]

In [16]:
first_cell(nb)

'# This is a Test NB\n<!--- @wandbcode{tracker} -->'

In [17]:
fancy_str = "@wandbcode"
search_string_in_nb(nb, fancy_str, cell_type=CellType.md)

True

In [18]:
tracker = "<!--- @wandbcode{tracker_id} -->"

In [19]:
import re
tracker

'<!--- @wandbcode{tracker_id} -->'

In [20]:
def get_tracker(cell):
    "Get the value inside <!--- @wandbcode{tracker} -->"
    if "@wandbcode" not in cell:
        return "[red]No Tracker[/red]"
    return re.search(r"@wandbcode{(.*?)}", cell).group(1)

In [87]:
def remove_rich_format(text):
    "Remove rich fancy coloring"
    res = re.search(r"\[(.*)\](.*?)\[(.*)\]", text)
    if res is None:
        return text
    else:
        return res.group(1)

In [90]:
remove_rich_format("[red]text[\red]")

'red'

In [91]:
remove_rich_format("ahjsod")

'ahjsod'

In [21]:
get_tracker("cacuca")

'[red]No Tracker[/red]'

In [22]:
get_tracker(first_cell(nb))

'tracker'

## Search tracker and libs

In [23]:
from rich import print
from rich.console import Console

from nb_helpers.utils import *
from nb_helpers.run import *

In [24]:
ex_nbs = Path.home()/'wandb/examples/'

In [25]:
notebooks = find_nbs(ex_nbs)

In [26]:
console = Console(width=200)

In [27]:
WANDB_FEATURES = ["Table", "sweep", "WandbCallback", "WandbLogger", "Artifact"]
PYTHON_LIBS = ["torch", "keras", "tensorflow", "sklearn", "yolo", "jax", "pandas", "numpy", "spacy", "transformers", "lightning", "fastai"]

In [28]:
def search_code(nb, features=WANDB_FEATURES):
    present_features = []
    for feat in listify(features):
        if search_string_in_nb(nb, feat):
            present_features.append(feat)
    return present_features


In [29]:
table = create_table(["#", "nb name", "tracker", "wandb features", "python libs"])

for i, nb_path in enumerate(notebooks):
    nb = read_nb(nb_path)
    tracker_id = get_wandb_tracker(nb)
    fname = nb_path.relative_to(Path.home()/"wandb/examples")
    features = search_code(nb)
    libs = search_code(nb, PYTHON_LIBS)
    table.add_row(f'{i+1}', str(fname), f'[green]{tracker_id}[/green]', ", ".join(features), ", ".join(libs))

In [30]:
console.print(table)

┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ #  ┃ nb name                                                            ┃ tracker                    ┃ wandb features                        ┃ python libs                                           ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1  │ colabs/Grouping_Feature.ipynb                                      │ No Tracker                 │                                       │                                                       │
│ 2  │ colabs/keras/Keras_param_opti_using_sweeps.ipynb                   │ sweeps-keras               │ sweep, WandbCallback                  │ keras, tensorflow, numpy                              │
│ 3  │ colabs/keras/Simple_Keras_Integration.ipynb                        │ keras-video                │ WandbCallback                         │ keras, tensorflow, pandas, numpy                      │
│ 4  │ colabs/wandb-artifacts/Basic_Artifacts_with_W&B.ipynb              │ No Tracker                 │ WandbCallback, Artifact               │ keras, tensorflow, sklearn, numpy                     │
│ 5  │ colabs/wandb-artifacts/Pipeline_Versioning_with_W&B_Artifacts.ipy… │ artifacts-video            │ WandbCallback, Artifact               │ keras, tensorflow, numpy                              │
│ 6  │ colabs/wandb-artifacts/Pipeline_Versioning_with_W&B_Artifacts_(Py… │ No Tracker                 │ Artifact                              │ torch                                                 │
│ 7  │ colabs/wandb-artifacts/W&B_artifacts_for_auditing_purposes.ipynb   │ audit-artifacts-colab      │ Artifact                              │ torch                                                 │
│ 8  │ colabs/wandb-artifacts/Artifacts_Quickstart_with_W&B.ipynb         │ artifacts_quickstart       │ WandbCallback, Artifact               │ keras, tensorflow, sklearn, numpy                     │
│ 9  │ colabs/tables/Log_Tables_Incrementally.ipynb                       │ No Tracker                 │ Table, Artifact                       │                                                       │
│ 10 │ colabs/tables/W&B_Tables_Quickstart.ipynb                          │ tables_quickstart          │ Table                                 │ sklearn, pandas, numpy                                │
│ 11 │ colabs/tables/AlphaFold_with_W&B_Align,_Fold,_Log.ipynb            │ alphafold                  │ Table                                 │ jax, numpy                                            │
│ 12 │ colabs/pycaret/Default_Credit_Prediction_Using_W&B_Pycaret_FastAP… │ No Tracker                 │ Table, Artifact                       │ sklearn, pandas, numpy                                │
│ 13 │ colabs/intro/Intro_to_Weights_&_Biases.ipynb                       │ intro-colab                │ WandbCallback                         │ keras, tensorflow, numpy                              │
│ 14 │ colabs/intro/3_in_1_Intro_to_Weights_&_Biases_CV,_NLP_and_RL.ipynb │ intro-colab-3-in-1         │ Artifact                              │ torch, keras, tensorflow, pandas, numpy, transformers │
│ 15 │ colabs/dsviz/W&B_Dataset_and_Predictions_Viz_Demo.ipynb            │ No Tracker                 │ Table, sweep, WandbCallback, Artifact │ numpy, fastai                                         │
│ 16 │ colabs/raytune/W&B_+_RayTune.ipynb                                 │ No Tracker                 │ WandbLogger                           │ torch, numpy                                          │
│ 17 │ colabs/datasets-predictions/Visualize_Model_Predictions_over_Time… │ No Tracker                 │ Table, Artifact                       │ torch, numpy                        

In [35]:
table

┏━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ #  ┃ nb name             ┃ tracker             ┃ wandb features     ┃ python libs         ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 1  │ colabs/Grouping_Fe… │ No Tracker          │                    │                     │
│ 2  │ colabs/keras/Keras… │ sweeps-keras        │ sweep,             │ keras, tensorflow,  │
│    │                     │                     │ WandbCallback      │ numpy               │
│ 3  │ colabs/keras/Simpl… │ keras-video         │ WandbCallback      │ keras, tensorflow,  │
│    │                     │                     │                    │ pandas, numpy       │
│ 4  │ colabs/wandb-artif… │ No Tracker          │ WandbCallback,     │ keras, tensorflow,  │
│    │                     │                     │ Artifact           │ sklearn, numpy      │
│ 5  │ colabs/wandb-artif… │ artifacts-video     │ WandbCallback,     │ keras, tensorflow,  │
│    │                     │                     │ Artifact           │ numpy               │
│ 6  │ colabs/wandb-artif… │ No Tracker          │ Artifact           │ torch               │
│ 7  │ colabs/wandb-artif… │ audit-artifacts-co… │ Artifact           │ torch               │
│ 8  │ colabs/wandb-artif… │ artifacts_quicksta… │ WandbCallback,     │ keras, tensorflow,  │
│    │                     │                     │ Artifact           │ sklearn, numpy      │
│ 9  │ colabs/tables/Log_… │ No Tracker          │ Table, Artifact    │                     │
│ 10 │ colabs/tables/W&B_… │ tables_quickstart   │ Table              │ sklearn, pandas,    │
│    │                     │                     │                    │ numpy               │
│ 11 │ colabs/tables/Alph… │ alphafold           │ Table              │ jax, numpy          │
│ 12 │ colabs/pycaret/Def… │ No Tracker          │ Table, Artifact    │ sklearn, pandas,    │
│    │                     │                     │                    │ numpy               │
│ 13 │ colabs/intro/Intro… │ intro-colab         │ WandbCallback      │ keras, tensorflow,  │
│    │                     │                     │                    │ numpy               │
│ 14 │ colabs/intro/3_in_… │ intro-colab-3-in-1  │ Artifact           │ torch, keras,       │
│    │                     │                     │                    │ tensorflow, pandas, │
│    │                     │                     │                    │ numpy, transformers │
│ 15 │ colabs/dsviz/W&B_D… │ No Tracker          │ Table, sweep,      │ numpy, fastai       │
│    │                     │                     │ WandbCallback,     │                     │
│    │                     │                     │ Artifact           │                     │
│ 16 │ colabs/raytune/W&B… │ No Tracker          │ WandbLogger        │ torch, numpy        │
│ 17 │ colabs/datasets-pr… │ No Tracker          │ Table, Artifact    │ torch, numpy        │
│ 18 │ colabs/datasets-pr… │ tables_whalesong    │ Table              │ tensorflow, numpy   │
│ 19 │ colabs/datasets-pr… │ tables_quickstart   │ Table              │ torch, numpy        │
│ 20 │ colabs/datasets-pr… │ No Tracker          │ Table, Artifact    │ numpy               │
│ 21 │ colabs/datasets-pr… │ tables_mendeleev    │ Table,             │ keras, tensorflow,  │
│    │                     │                     │ WandbCallback,     │ numpy               │
│    │                     │                     │ Artifact           │                     │
│ 22 │ colabs/fastai/Weig… │ No Tracker          │ WandbCallback      │ fastai              │
│ 23 │ colabs/deepchem/W&… │ deepchem            │ Table, WandbLogger │ keras, tensorflow,  │
│    │                     │                     │                    │ numpy, transformers │
│ 24 │ colabs/tensorflow/… │ tf-sweeps           │ sweep,             │ keras, tensorflow,  │
│    │                     │                     │ W

In [34]:
summary_nbs(path=ex_nbs)

CONSOLE.is_terminal(): False

Reading 67 notebooks

┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ #  ┃ nb name                                           ┃ tracker                    ┃ wandb features                        ┃ python libs                                        ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1  │ wandb/examples/colabs/Grouping_Feature.ipynb      │ No Tracker                 │                                       │                                                    │
│ 2  │ wandb/examples/colabs/keras/Keras_param_opti_usi… │ sweeps-keras               │ sweep, WandbCallback                  │ keras, tensorflow, numpy                           │
│ 3  │ wandb/examples/colabs/keras/Simple_Keras_Integra… │ keras-video                │ WandbCallback                         │ keras, tensorflow, pandas, numpy                   │
│ 4  │ wandb/examples/colabs/wandb-artifacts/Basic_Arti… │ No Tracker                 │ WandbCallback, Artifact               │ keras, tensorflow, sklearn, numpy                  │
│ 5  │ wandb/examples/colabs/wandb-artifacts/Pipeline_V… │ artifacts-video            │ WandbCallback, Artifact               │ keras, tensorflow, numpy                           │
│ 6  │ wandb/examples/colabs/wandb-artifacts/Pipeline_V… │ No Tracker                 │ Artifact                              │ torch                                              │
│ 7  │ wandb/examples/colabs/wandb-artifacts/W&B_artifa… │ audit-artifacts-colab      │ Artifact                              │ torch                                              │
│ 8  │ wandb/examples/colabs/wandb-artifacts/Artifacts_… │ artifacts_quickstart       │ WandbCallback, Artifact               │ keras, tensorflow, sklearn, numpy                  │
│ 9  │ wandb/examples/colabs/tables/Log_Tables_Incremen… │ No Tracker                 │ Table, Artifact                       │                                                    │
│ 10 │ wandb/examples/colabs/tables/W&B_Tables_Quicksta… │ tables_quickstart          │ Table                                 │ sklearn, pandas, numpy                             │
│ 11 │ wandb/examples/colabs/tables/AlphaFold_with_W&B_… │ alphafold                  │ Table                                 │ jax, numpy                                         │
│ 12 │ wandb/examples/colabs/pycaret/Default_Credit_Pre… │ No Tracker                 │ Table, Artifact                       │ sklearn, pandas, numpy                             │
│ 13 │ wandb/examples/colabs/intro/Intro_to_Weights_&_B… │ intro-colab                │ WandbCallback                         │ keras, tensorflow, numpy                           │
│ 14 │ wandb/examples/colabs/intro/3_in_1_Intro_to_Weig… │ intro-colab-3-in-1         │ Artifact                              │ torch, keras, tensorflow, pandas, numpy,           │
│    │                                                   │                            │                                       │ transformers                                       │
│ 15 │ wandb/examples/colabs/dsviz/W&B_Dataset_and_Pred… │ No Tracker                 │ Table, sweep, WandbCallback, Artifact │ numpy, fastai                                      │
│ 16 │ wandb/examples/colabs/raytune/W&B_+_RayTune.ipynb │ No Tracker                 │ WandbLogger                           │ torch, numpy                                       │
│ 17 │ wandb/examples/colabs/datasets-predictions/Visua… │ No Tracker                 │ Table, Artifact                       │ torch, numpy                                       │
│ 18 │ wandb/examples/colabs/datasets-predictions/Loggi… │ tables_whalesong           │ Table                                 │ tensorflow, numpy                                  │
│ 19 │ wandb/examp

END!

In [ ]:
table.